In [1]:
import os
import math
import pickle
import json
import numpy as np
import pandas as pd
import networkx as nx
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, Input
from sklearn.metrics import mean_absolute_error, mean_squared_error, precision_score, recall_score, f1_score
from scipy.stats import pearsonr
from sklearn.utils import class_weight
from tqdm import tqdm

# Cấu hình GPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# ==============================================================================
# 1. CONSTANTS & SCALERS (Trích xuất từ Config DSTGCN gốc)
# ==============================================================================
# Các giá trị này lấy từ cell config trong notebook dstgcn (1).ipynb
SPATIAL_MEAN = np.array([6.40254292e+00, 4.81553347e+00, 2.64811501e+01, 5.54260681e+00, 
                         7.84915710e+00, 9.01424670e+00, 1.05751666e+00, 2.83014290e+01, 
                         1.77408007e+01, 3.24747843e+01, 1.05794121e+00, 3.47343126e+00, 
                         6.26610390e+00, 3.39390529e+00, 1.28002351e+01, 5.71796174e-02, 
                         2.36710361e-02, 5.17406913e-02, 8.18431583e-01, 4.57574675e-01, 
                         1.61370302e+02, 2.00511484e+00])

SPATIAL_STD = np.array([8.25421040e+00, 6.98772605e+00, 3.12181863e+01, 7.78804699e+00, 
                        1.12482891e+01, 8.86203753e+00, 4.88544842e+00, 4.65677369e+01, 
                        1.88312515e+01, 6.70090744e+01, 2.95457633e+00, 4.29173854e+00, 
                        9.78727429e+00, 3.76843985e+00, 1.19405955e+01, 2.71230076e-01, 
                        1.88342431e-01, 2.82777835e-01, 2.18729099e+00, 1.55007715e+00, 
                        2.04879612e+02, 7.87830458e-02])

# Tốc độ (Temporal features)
TEMPORAL_MEAN = np.array([18.54884516])
TEMPORAL_STD = np.array([19.87195773])

# ==============================================================================
# 2. UTILS & COORDINATE CONVERSION
# ==============================================================================
x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626
a = 6378245.0
ee = 0.00669342162296594323

def gcj02_to_wgs84(lng, lat):
    # (Giữ nguyên logic convert tọa độ của bạn để đảm bảo khớp dữ liệu)
    dlat = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + 0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    dlat += (20.0 * math.sin(6.0 * lng * pi) + 20.0 * math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    dlat += (20.0 * math.sin(lat * pi) + 40.0 * math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    dlat += (160.0 * math.sin(lat / 12.0 * pi) + 320 * math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    
    dlng = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + 0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    dlng += (20.0 * math.sin(6.0 * lng * pi) + 20.0 * math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    dlng += (20.0 * math.sin(lng * pi) + 40.0 * math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    dlng += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 * math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng, lat * 2 - mglat]

LONGITUDE_MIN_RAW, LATITUDE_MIN_RAW = 116.09608, 39.69086
LONGITUDE_MIN, LATITUDE_MIN = gcj02_to_wgs84(LONGITUDE_MIN_RAW, LATITUDE_MIN_RAW)
WIDTH_SINGLE = 0.01 / math.cos(LATITUDE_MIN / 180 * math.pi) / 5
HEIGHT_SINGLE = 0.01 / 5

# ==============================================================================
# 3. ADVANCED DATA LOADER
# ==============================================================================
class DSTGCN_Loader_Enhanced(keras.utils.Sequence):
    def __init__(self, data_dir, mode='train', batch_size=32, k_order=2, max_nodes=20, shuffle=True, **kwargs):
        # FIX 1: Gọi super().__init__ để Keras không báo warning
        super().__init__(**kwargs) 
        self.batch_size = batch_size
        self.k_order = k_order
        self.max_nodes = max_nodes
        self.shuffle = shuffle
        
        print(f"[{mode.upper()}] Loading data...")
        with open(os.path.join(data_dir, "beijing_roadnet.gpickle"), 'rb') as f:
            self.G = pickle.load(f)
        self.nodes_df = pd.read_hdf(os.path.join(data_dir, "edges_data.h5"))
        self.accident_df = pd.read_hdf(os.path.join(data_dir, "accident.h5"), key=mode)
        
        # Load speed & Fillna
        speed_df = pd.read_hdf(os.path.join(data_dir, "all_grids_speed.h5"))
        if not isinstance(speed_df.index, pd.DatetimeIndex):
            speed_df.index = pd.to_datetime(speed_df.index)
        self.speed_df = speed_df.resample("1h").mean().interpolate(method='time').fillna(0.0)
        
        self.indices = np.arange(len(self.accident_df))
        if self.shuffle: np.random.shuffle(self.indices)
        
        # Pre-calculate grid mapping
        self.node_grid_map = {}
        for n_id, row in self.nodes_df.iterrows():
            x_id = int(np.floor((row['XCoord'] - LONGITUDE_MIN) / WIDTH_SINGLE))
            y_id = int(np.floor((row['YCoord'] - LATITUDE_MIN) / HEIGHT_SINGLE))
            self.node_grid_map[n_id] = f"{y_id},{x_id}"

    def __len__(self):
        return int(np.floor(len(self.accident_df) / self.batch_size))
        
    def normalize_adj(self, adj):
        adj = adj + np.eye(adj.shape[0])
        row_sum = np.sum(adj, axis=1)
        d_inv_sqrt = np.power(row_sum, -0.5)
        d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
        d_mat_inv_sqrt = np.diag(d_inv_sqrt)
        return np.dot(np.dot(d_mat_inv_sqrt, adj), d_mat_inv_sqrt)

    def __getitem__(self, index):
        batch_indices = self.indices[index*self.batch_size : (index+1)*self.batch_size]
        
        batch_A = np.zeros((self.batch_size, self.max_nodes, self.max_nodes), dtype=np.float32)
        batch_F_temporal = np.zeros((self.batch_size, 24, self.max_nodes, 1), dtype=np.float32) 
        batch_F_spatial = np.zeros((self.batch_size, self.max_nodes, 22), dtype=np.float32)     
        batch_Y = np.zeros((self.batch_size, 1), dtype=np.float32)
        
        for i, idx in enumerate(batch_indices):
            row = self.accident_df.iloc[idx]
            
            # FIX 2: Dùng .iloc để truy cập vị trí, tránh FutureWarning
            accident_time = row.iloc[2]
            target_node = row.iloc[3]
            label = row.iloc[4]
            
            try: neighbors = nx.single_source_shortest_path_length(self.G, target_node, cutoff=self.k_order)
            except: neighbors = {target_node: 0}
            
            valid_nodes = ([target_node] + [n for n in sorted(neighbors.keys()) if n != target_node])[:self.max_nodes]
            num_valid = len(valid_nodes)
            
            subg = self.G.subgraph(valid_nodes)
            adj = nx.to_numpy_array(subg, nodelist=valid_nodes)
            batch_A[i, :num_valid, :num_valid] = self.normalize_adj(adj)
            
            spatial_raw = np.array(self.nodes_df.loc[valid_nodes]['spatial_features'].tolist())
            if len(spatial_raw) > 0:
                spatial_norm = (spatial_raw - SPATIAL_MEAN) / SPATIAL_STD
                batch_F_spatial[i, :num_valid, :] = spatial_norm

            time_range = pd.date_range(end=accident_time.strftime("%Y%m%d %H"), freq="1h", periods=24)
            
            for n_idx, node_id in enumerate(valid_nodes):
                col = self.node_grid_map.get(node_id)
                if col and col in self.speed_df.columns:
                    vals = self.speed_df[col].reindex(time_range, fill_value=0.0).values
                    vals_norm = (vals - TEMPORAL_MEAN) / TEMPORAL_STD
                    batch_F_temporal[i, :, n_idx, 0] = vals_norm
            
            batch_Y[i, 0] = label

        batch_A_T = np.repeat(batch_A[:, np.newaxis, :, :], 24, axis=1)
        
        inputs = {
            'A': batch_A,           
            'A_T': batch_A_T,       
            'F_T': batch_F_temporal,
            'F_S': batch_F_spatial  
        }
        return inputs, batch_Y
    
    def on_epoch_end(self):
        if self.shuffle: np.random.shuffle(self.indices)

# ==============================================================================
# 4. CUSTOM LOSS: PEARSON LOSS (KEY CHO PCC > 0.8)
# ==============================================================================
def pearson_correlation_loss(y_true, y_pred):
    """
    Loss function để tối đa hóa Pearson Correlation.
    Loss = 1 - PCC. 
    """
    x = y_true
    y = y_pred
    mx = tf.reduce_mean(x)
    my = tf.reduce_mean(y)
    xm, ym = x - mx, y - my
    r_num = tf.reduce_sum(xm * ym)
    r_den = tf.sqrt(tf.reduce_sum(tf.square(xm)) * tf.reduce_sum(tf.square(ym)) + 1e-8) # + epsilon
    r = r_num / r_den
    return 1.0 - r # Minimize (1 - PCC) <=> Maximize PCC

def hybrid_loss(y_true, y_pred):
    # Kết hợp BCE để học phân loại và Pearson để học ranking
    # Trọng số 0.5 - 0.5 hoặc tùy chỉnh
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    pcc_loss = pearson_correlation_loss(y_true, y_pred)
    return bce + 0.5 * pcc_loss

# ==============================================================================
# 5. MODEL ARCHITECTURE: ENHANCED MG-TAR
# ==============================================================================
class ResidualGCN(layers.Layer):
    def __init__(self, units, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
        self.dense = layers.Dense(units)
        self.bn = layers.BatchNormalization() # Thêm BN để ổn định
    
    def call(self, inputs):
        features, adj = inputs
        # GCN Operation: A * X * W
        support = self.dense(features)
        output = tf.matmul(adj, support)
        
        # Residual Connection nếu chiều khớp
        if features.shape[-1] == self.units:
            output = output + features
            
        output = self.bn(output)
        return self.activation(output)

def build_enhanced_model(max_nodes=20, time_steps=24):
    # Inputs
    A = Input(shape=(max_nodes, max_nodes), name='A')
    A_T = Input(shape=(time_steps, max_nodes, max_nodes), name='A_T')
    F_T = Input(shape=(time_steps, max_nodes, 1), name='F_T') # Speed
    F_S = Input(shape=(max_nodes, 22), name='F_S') # Spatial features
    
    # 1. Spatial Embedding (Xử lý F_S riêng)
    s_emb = layers.Dense(16, activation='relu')(F_S)
    
    # 2. Temporal Processing Loop
    gcn_units = 32
    gcn_layer = ResidualGCN(gcn_units, activation='relu')
    
    H_sequence = []
    
    for t in range(time_steps):
        # Lấy features tại t: (Batch, Nodes, 1)
        ft = layers.Lambda(lambda x: x[:, t, :, :])(F_T)
        at = layers.Lambda(lambda x: x[:, t, :, :])(A_T)
        
        # Concatenate Speed (ft) với Spatial Embedding (s_emb) -> (Batch, Nodes, 17)
        combined_feat = layers.Concatenate()([ft, s_emb])
        
        # GCN spatial view
        h_s = gcn_layer([combined_feat, A])
        # GCN temporal view (dùng A_T - thực ra là lặp lại A nhưng logic MG-TAR có thể mở rộng)
        h_t = gcn_layer([combined_feat, at])
        
        # Merge views
        h_merge = layers.Concatenate()([h_s, h_t]) # (Batch, Nodes, 64)
        
        # Pooling nodes -> (Batch, 64)
        pool = layers.GlobalAveragePooling1D()(h_merge)
        H_sequence.append(pool)
        
    # Stack -> (Batch, 24, 64)
    H_seq = layers.Lambda(lambda x: tf.stack(x, axis=1))(H_sequence)
    
    # 3. Sequence Modeling: Bi-GRU (Thay vì GRU thường)
    x = layers.Bidirectional(layers.GRU(64, return_sequences=True))(H_seq)
    x = layers.Bidirectional(layers.GRU(32))(x)
    
    # 4. Final Prediction
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=[A, A_T, F_T, F_S], outputs=output)
    
    # Optimizer & Compile
    # Sử dụng Hybrid Loss
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=hybrid_loss,
                  metrics=['mae', 'accuracy'])
    return model

# ==============================================================================
# 6. MAIN EXECUTION
# ==============================================================================
if __name__ == "__main__":
    DATA_DIR = "/kaggle/input/dstgcn-dataset" 
    BATCH_SIZE = 64 # Tăng batch size để gradient ổn định hơn
    
    if os.path.exists(DATA_DIR):
        # Loaders
        train_gen = DSTGCN_Loader_Enhanced(DATA_DIR, 'train', BATCH_SIZE)
        val_gen = DSTGCN_Loader_Enhanced(DATA_DIR, 'validate', BATCH_SIZE)
        test_gen = DSTGCN_Loader_Enhanced(DATA_DIR, 'test', BATCH_SIZE, shuffle=False)
        
        # Class Weights
        print("Calculating class weights...")
        labels = train_gen.accident_df.iloc[:, 4].values
        cw = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels)
        cw_dict = dict(enumerate(cw))
        print(f"Weights: {cw_dict}")
        
        # Build Model
        model = build_enhanced_model()
        model.summary()
        
        # Callbacks
        es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
        
        # Train
        print("\n--- START TRAINING WITH PEARSON OPTIMIZATION ---")
        history = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=50, 
            class_weight=cw_dict,
            callbacks=[es, rlr],
            verbose=1
        )
        
        # Evaluate
        print("\n--- FINAL EVALUATION ---")
        y_true, y_pred = [], []
        
        for i in tqdm(range(len(test_gen)), desc="Predicting"):
            inputs, y = test_gen[i]
            preds = model.predict_on_batch(inputs)
            y_true.extend(y.flatten())
            y_pred.extend(preds.flatten())
            
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mae = mean_absolute_error(y_true, y_pred)
        pcc, _ = pearsonr(y_true, y_pred)
        
        print(f"\nRMSE: {rmse:.4f}")
        print(f"MAE : {mae:.4f}")
        print(f"PCC : {pcc:.4f} (Target > 0.8)")
        
    else:
        print("Dataset not found!")

2025-12-30 02:56:02.213243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767063362.234863     705 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767063362.241470     705 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767063362.258433     705 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767063362.258457     705 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767063362.258460     705 computation_placer.cc:177] computation placer alr

[TRAIN] Loading data...
[VALIDATE] Loading data...
[TEST] Loading data...
Calculating class weights...
Weights: {0: np.float64(1.0), 1: np.float64(1.0)}


I0000 00:00:1767063437.257920     705 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ F_T (InputLayer)    │ (None, 24, 20, 1) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ F_S (InputLayer)    │ (None, 20, 22)    │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 20, 16)    │        368 │ F_S[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ A_T (InputLayer)    │ (None, 24, 20,    │          0 │ -                 │
│                     │ 20)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_4 (Lambda)   │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_6 (Lambda)   │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_8 (Lambda)   │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_10 (Lambda)  │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_12 (Lambda)  │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_14 (Lambda)  │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_16 (Lambda)  │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_18 (Lambda)  │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_20 (Lambda)  │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_22 (Lambda)  │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_24 (Lambda)  │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_26 (Lambda)  │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_28 (Lambda)  │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_30 (Lambda)  │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_32 (Lambda)  │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_34 (Lambda)  │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_36 (Lambda)  │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_38 (Lambda)  │ (None, 20, 1)     │          0 │ F_T[0][0]         │
├─────────────────────┼───────────────────┼────────────┼─────────────────

...


--- START TRAINING WITH PEARSON OPTIMIZATION ---
Epoch 1/50


I0000 00:00:1767063455.148978     747 cuda_dnn.cc:529] Loaded cuDNN version 91002


139/139 ━━━━━━━━━━━━━━━━━━━━ 53s 260ms/step - accuracy: 0.6784 - loss: 0.9007 - mae: 0.4210 - val_accuracy: 0.7625 - val_loss: 0.7030 - val_mae: 0.3411 - learning_rate: 0.0010
Epoch 2/50
139/139 ━━━━━━━━━━━━━━━━━━━━ 35s 248ms/step - accuracy: 0.7603 - loss: 0.7235 - mae: 0.3381 - val_accuracy: 0.7789 - val_loss: 0.6521 - val_mae: 0.3118 - learning_rate: 0.0010
Epoch 3/50
139/139 ━━━━━━━━━━━━━━━━━━━━ 35s 248ms/step - accuracy: 0.7804 - loss: 0.6689 - mae: 0.3106 - val_accuracy: 0.7828 - val_loss: 0.6332 - val_mae: 0.3004 - learning_rate: 0.0010
Epoch 4/50
139/139 ━━━━━━━━━━━━━━━━━━━━ 35s 250ms/step - accuracy: 0.7844 - loss: 0.6409 - mae: 0.2995 - val_accuracy: 0.7867 - val_loss: 0.6333 - val_mae: 0.3032 - learning_rate: 0.0010
Epoch 5/50
139/139 ━━━━━━━━━━━━━━━━━━━━ 34s 245ms/step - accuracy: 0.7833 - loss: 0.6296 - mae: 0.2951 - val_accuracy: 0.7852 - val_loss: 0.6488 - val_mae: 0.2822 - learning_rate: 0.0010
Epoch 6/50
139/139 ━━━━━━━━━━━━━━━━━━━━ 34s 246ms/step - accuracy: 0.7880 - 

Predicting: 100%|██████████| 39/39 [00:08<00:00,  4.61it/s]


RMSE: 0.3416
MAE : 0.2315
PCC : 0.7310 (Target > 0.8)
